In [ ]:
# imports
import shutil
import numpy as np
import pandas as pd
import os
import math
import pickle
import sqlalchemy
from sqlalchemy.orm import sessionmaker
from scipy import optimize
from scipy.stats import sem
import subprocess

def check_repeated_constructs(x, index_loc):
    '''
    Helper function, Returns location of counts with respect to study conditions/replicate names.
    '''
    if len(x) < max(index_loc):
        sub = index_loc[index_loc < len(x)]
        return(x[sub])
    else:
        return(x[index_loc])
    
def get_raw_counts(curr_counts):
    #display(curr_counts) 
    '''
    Helper function, gets the raw counts based on the T0 and TEnd annotations of the sample names
    '''
    print('Getting raw counts...')
    # get counts
    T0_counts = curr_counts['T0_counts'].apply(    
        lambda x: np.array(x.split(";"), dtype = np.float64)
    )

    T0_counts = pd.DataFrame(data = T0_counts.tolist(),
                   index = T0_counts.index, columns = curr_counts['T0_replicate_names'].iloc[0].split(';'))

    TEnd_counts = curr_counts['TEnd_counts'].apply(    
        lambda x: np.array(x.split(";"), dtype = np.float64)
    )

    TEnd_counts = pd.DataFrame(data = TEnd_counts.tolist(),
                   index = TEnd_counts.index, columns = curr_counts['TEnd_replicate_names'].iloc[0].split(';'))
    
    # make sure no columns are filled with NAs completely (in case of additional annotations)
    NA_replicate = T0_counts.isna().sum()
    if (NA_replicate == T0_counts.shape[0]).sum() > 0:
        print('Removing NA replicate from T0...')
        T0_counts.drop(NA_replicate.index[NA_replicate == T0_counts.shape[0]], axis = 1, inplace = True)
    
    NA_replicate = TEnd_counts.isna().sum()
    if (NA_replicate == TEnd_counts.shape[0]).sum() > 0:
        print('Removing NA replicate from TEnd...')
        TEnd_counts.drop(NA_replicate.index[NA_replicate == TEnd_counts.shape[0]], axis = 1, inplace = True)

    T0_counts = T0_counts.fillna(0)
    TEnd_counts = TEnd_counts.fillna(0)
    
    return((T0_counts, TEnd_counts))

def run_horlbeck_score(curr_counts, curr_study, curr_cl, do_preprocessing = True, store_loc = os.getcwd(), save_dir = 'HORLBECK_Files', re_run = False, filterThreshold = 35):
    '''
    
    Calculates Horlbeck score. Score files will created at the designated store location and save directory. 

    **Params**:
    * curr_counts: Counts to calculate scores to.)
    * curr_study: String, name of study to analyze data for.
    * curr_cl: String, name of cell line to analyze data for.
    * store_loc: String: Directory to store the MAGeCK files to. (Default: current working directory)
    * save_dir: String: Folder name to store the MAGeCK files to. (Default: 'Horlbeck_Files')
    * do_preprocessing: Boolean. Run Horlbeck preprocessing (Default: True)
    * re_run: Boolean. Recreate and rerun the results instead of loading for subsequent analyses (Default: False)

    **Returns**:

    * horlbeck_res: A dict that contains a pandas dataframe for Horlbeck Score.

    '''
    print('Running horlbeck score...')
    
    ######### preprocessing
    
    print('Running preprocessing...')

    if do_preprocessing:
        curr_counts = run_horlbeck_preprocessing(curr_counts, filterThreshold)


    #########/ preprocessing
    
    # get save location
    save_loc = os.path.join(store_loc, save_dir, curr_study, curr_cl)
    os.makedirs(save_loc, exist_ok = True)
    
    ######### original horlbeck scoring

    # first, drop the rows with nan replicateFCname
    curr_counts.dropna(subset = ['FC_Averaged_abbaAveraged'], inplace = True)

    # get ab/ba
    a_average, b_average = curr_counts.loc[curr_counts['target_type'] != 'Dual'].copy(), curr_counts.loc[curr_counts['target_type'] != 'Dual'].copy()
    #curr_counts.loc[curr_counts['target_type'] == 'Single'].copy(), curr_counts.loc[curr_counts['target_type'] == 'Single'].copy()
    #
    a_average = a_average[a_average['sgRNA_target_name_g2'] == "CONTROL"]
    b_average = b_average[b_average['sgRNA_target_name_g1'] == "CONTROL"]

    a_average = a_average.groupby('sgRNA_guide_name_g1')['FC_Averaged_abbaAveraged'].apply(np.mean)
    b_average = b_average.groupby('sgRNA_guide_name_g2')['FC_Averaged_abbaAveraged'].apply(np.mean)

    # single, control, and dual phenotypes are used in calculation
    all_pairs = set(curr_counts['sgRNA_guide_name_g1']).union(set(curr_counts['sgRNA_guide_name_g2']))
    curr_counts['GI_Averaged'] = 0

    # for missing pairs, update a_average, b_average
    a_average_0s = list(all_pairs.difference(set(a_average.index)))
    a_average = pd.concat([a_average, pd.Series(data = np.zeros(len(a_average_0s)), index = a_average_0s)])

    b_average_0s = list(all_pairs.difference(set(b_average.index)))
    b_average = pd.concat([b_average, pd.Series(data = np.zeros(len(b_average_0s)), index = b_average_0s)])

    # store in a matrix
    GI_Score_1 = pd.DataFrame(0, index = sorted(list(all_pairs)), columns = sorted(list(all_pairs)), dtype = np.float64)
    GI_Score_2 = pd.DataFrame(0, index = sorted(list(all_pairs)), columns = sorted(list(all_pairs)), dtype = np.float64)
    
    
    # scores have already been computed
    if os.path.exists(os.path.join(save_loc, "GI_Score_1.gzip")) and (not re_run):
        print('Scores exist For GI_Score_1! Loading...')
        GI_Score_1 = pd.read_pickle(os.path.join(save_loc, "GI_Score_1.gzip"))
    else:
        print('Calculating GI_Score_1...')
        
        ## A orientation ()

        ## go through all query sgRNAs
        for query_sgRNA in all_pairs:

            ## get all the pairs with the given query
            idx_loc = (curr_counts['sgRNA_guide_name_g2'] == query_sgRNA)
            #print(query_sgRNA)
            #print(idx_loc)

            if len(idx_loc) == 0:
                continue

            ## all pairs 
            curr_filtered_pairs = curr_counts.loc[idx_loc, :]

            ## get sgRNAs assayed together with the query sgRNA
            selected_sgRNAs = curr_filtered_pairs['sgRNA_guide_name_g1'].values

            if 'Control' in set(curr_counts['target_type']):
                #print("In Control")
                control_sgRNAs = np.where(curr_filtered_pairs['sgRNA_target_name_g1'] == "CONTROL")[0]
                #print(control_sgRNAs)
                #print(len(control_sgRNAs))

            # Fit to a quadratic formula, where the x is the single phenotypes and y is the pair phenotypes

            xs = a_average.loc[selected_sgRNAs].values # a -> b
            ys = curr_filtered_pairs['FC_Averaged_abbaAveraged'].values
            bs = b_average.loc[query_sgRNA] # b -> a

            res_fn = quadFitForceIntercept(xs, ys, bs)

            # get expected
            expected_phenotype = res_fn(xs)

            # the difference is the GI score
            GI_Score = ys - expected_phenotype

            if ('Control' in set(curr_counts['target_type'])) and len(control_sgRNAs) > 0:
                if GI_Score[control_sgRNAs].std() != 0:
                    GI_Score /= GI_Score[control_sgRNAs].std()
                   
           
            #print(GI_Score_1.loc[query_sgRNA, selected_sgRNAs].shape)
            #print(len(GI_Score))
            GI_Score = GI_Score.astype('int32')
            GI_Score_1.loc[query_sgRNA, selected_sgRNAs] = GI_Score
            #print(GI_Score_1.loc[query_sgRNA, selected_sgRNAs])
        # save scores for future loading
      #  GI_Score_1.to_pickle(os.path.join(save_loc, "GI_Score_1.gzip"))
    
    if os.path.exists(os.path.join(save_loc, "GI_Score_2.gzip")) and (not re_run):
        print('Scores exist For GI_Score_2! Loading...')
        GI_Score_2 = pd.read_pickle(os.path.join(save_loc, "GI_Score_2.gzip"))
    else:
        print('Calculating GI_Score_2...')

        ## B orientation ()
        ## go through all query sgRNAs
        for query_sgRNA in all_pairs:

            ## get all the pairs with the given query
            idx_loc = (curr_counts['sgRNA_guide_name_g1'] == query_sgRNA)

            if len(idx_loc) == 0:
                continue

            ## all pairs 
            curr_filtered_pairs = curr_counts.loc[idx_loc, :]

            ## get sgRNAs assayed together with the query sgRNA
            selected_sgRNAs = curr_filtered_pairs['sgRNA_guide_name_g2'].values

            if 'Control' in set(curr_counts['target_type']):
                control_sgRNAs = np.where(curr_filtered_pairs['sgRNA_target_name_g2'] == "CONTROL")[0]

            # Fit to a quadratic formula, where the x is the single phenotypes and y is the pair phenotypes

            xs = b_average.loc[selected_sgRNAs].values # b -> a
            ys = curr_filtered_pairs['FC_Averaged_abbaAveraged'].values
            bs = a_average.loc[query_sgRNA] # a -> b

            res_fn = quadFitForceIntercept(xs, ys, bs)

            # get expected
            expected_phenotype = res_fn(xs)

            # the difference is the GI score
            GI_Score = ys - expected_phenotype

            if ('Control' in set(curr_counts['target_type'])) and len(control_sgRNAs) > 0:
                if GI_Score[control_sgRNAs].std() != 0:
                    GI_Score /= GI_Score[control_sgRNAs].std()

            # set the 
            #curr_res['sgRNA_level']['dual'].loc[idx_loc, replicate_GI_name] += GI_Score
            GI_Score_2.loc[query_sgRNA, selected_sgRNAs] = GI_Score


        # save scores for future loading
        GI_Score_2.to_pickle(os.path.join(save_loc, "GI_Score_2.gzip"))
    
    
    # average between A and B orientations
    #curr_res['sgRNA_level']['dual'][replicate_GI_name] /= 2
    GI_Score_avg = (GI_Score_1 + GI_Score_2)/2
    GI_Score_avg = (GI_Score_avg + GI_Score_avg.T)/2

    for i in range(len(curr_counts['GI_Averaged'])):
        guide_1 = curr_counts['sgRNA_guide_name_g1'].iloc[i]
        guide_2 = curr_counts['sgRNA_guide_name_g2'].iloc[i]

        curr_counts['GI_Averaged'].iloc[i] = GI_Score_avg.loc[guide_1, guide_2]

    
    ######### /original horlbeck scoring
    
    
    # store results
    SL_score = curr_counts.groupby('gene_pair')['GI_Averaged'].apply(lambda x: np.mean(x))
    SE = curr_counts.groupby('gene_pair')['GI_Averaged'].apply(lambda x: sem(x, ddof=1))

    genes_1 = [i.split('|')[0] for i in SL_score.index]
    genes_2 = [i.split('|')[1] for i in SL_score.index]
    
    horlbeck_results = pd.DataFrame(data = {'SL_score' : SL_score.values,
                                             'standard_error' : SE.values,
                                             'Gene 1' : genes_1,
                                             'Gene 2' : genes_2}, index = SL_score.index)
    
    
    # remove possible controls
    control_idx = np.array([True if 'CONTROL' in i else False for i in horlbeck_results.index])
    horlbeck_results = horlbeck_results.loc[~control_idx]
    
    results = {}
    results['HORLBECK_SCORE'] = horlbeck_results

    return(results)

def quadFitForceIntercept(xdata, ydata, bdata):
    m1 = optimize.fmin(lambda m, x, y: ((m[0]*(x**2) + m[1]*x + bdata - y)**2).sum(), x0=[0.1,0.1], args=(xdata, ydata), disp=0)
    
    return lambda x1: m1[0]*(np.array(x1)**2) + m1[1]*np.array(x1) + bdata


def run_horlbeck_preprocessing(curr_counts, filterThreshold = 35, pseudocount = 10):
        
    T0_counts, TEnd_counts = get_raw_counts(curr_counts.copy())
    
    # horlbeck uses single x single as double, proceed to move them to dual instead
    replace_idx = (curr_counts['target_type'] == 'Single') & (curr_counts['sgRNA_target_name_g1'] == curr_counts['sgRNA_target_name_g2'])
    curr_counts.loc[replace_idx, 'target_type'] = 'Dual'

    if T0_counts.shape[1] != TEnd_counts.shape[1]:
        print("Mismatch times, averaging...")

        T0_counts = pd.DataFrame(data = T0_counts.apply(lambda x: np.mean(x), axis = 1).values,
                             index = T0_counts.index)

        TEnd_counts = pd.DataFrame(data = TEnd_counts.apply(lambda x: np.mean(x), axis = 1).values,
                 index = TEnd_counts.index)

    T0_counts = pd.concat([T0_counts, curr_counts['sgRNA_guide_name_g1'], curr_counts['sgRNA_guide_name_g2']], axis = 1)
    TEnd_counts = pd.concat([TEnd_counts, curr_counts['sgRNA_guide_name_g1'], curr_counts['sgRNA_guide_name_g2']], axis = 1)
    all_sgRNAs = set(TEnd_counts['sgRNA_guide_name_g1']).union(set(TEnd_counts['sgRNA_guide_name_g2']))

    # add sorted targets
    sorted_gene_pairs, sorted_gene_guides = sort_pairs_and_guides(curr_counts.copy())
    curr_counts['sgRNA_pair'] = sorted_gene_guides
    curr_counts['gene_pair'] = sorted_gene_pairs
    
    replicate_list = []
    for replicate_i in range(len(T0_counts.columns)-2):
        print("For replicate " + str(replicate_i + 1))
        meanCounts = pd.concat((TEnd_counts.iloc[:,replicate_i].groupby(TEnd_counts['sgRNA_guide_name_g1']).agg(np.median),TEnd_counts.iloc[:,replicate_i].groupby(TEnd_counts['sgRNA_guide_name_g2']).agg(np.median)),axis=1, keys=['sgRNA_guide_name_g1', 'sgRNA_guide_name_g2'])
        sgsToFilter = set(meanCounts.loc[meanCounts.loc[:,'sgRNA_guide_name_g1'] < filterThreshold].index).union(set(meanCounts.loc[meanCounts.loc[:,'sgRNA_guide_name_g2'] < filterThreshold].index))
        print(" ".join(["Total of", str(len(sgsToFilter)), 'sgRNAs were filtered out of', str(len(all_sgRNAs))]))

        chosen_idx = np.array([True if i not in sgsToFilter else False for i in TEnd_counts['sgRNA_guide_name_g1']]) & np.array([True if i not in sgsToFilter else False for i in TEnd_counts['sgRNA_guide_name_g2']])
        TEnd_counts_curr = TEnd_counts.iloc[chosen_idx, replicate_i]
        T0_counts_curr = T0_counts.iloc[chosen_idx, replicate_i]

        counts_ratio = ((T0_counts_curr + pseudocount).sum()*1.0)/(TEnd_counts_curr + pseudocount).sum()

        # calculate FC like in horlbeck
        replicate_FC = np.log2((TEnd_counts_curr + pseudocount)/(T0_counts_curr + pseudocount)/counts_ratio)
        replicate_FC.columns = ['Replicate_' + str(replicate_i+1) + "_FC"]
        replicate_FC.name = 'Replicate_' + str(replicate_i+1) + "_FC"

        # get control
        control_effect = 0
        if 'Control' in set(curr_counts['target_type']):
            control_index = curr_counts['target_type'] == 'Control'
            if control_index.sum() != 0:
                control_effect = replicate_FC.loc[control_index].median()

        replicate_FC -= control_effect

        # doubling differences, taken from original code
        replicate_FC /= 6.3

        curr_counts = curr_counts.join(replicate_FC)

        replicate_list.append(replicate_FC)

    # save the results to original data
    replicate_list = pd.concat(replicate_list, axis = 1)
    replicate_list = replicate_list.dropna()

    replicate_list = replicate_list.mean(axis = 1)

    replicate_list.columns = ['FC_Averaged']
    replicate_list.name = 'FC_Averaged'

    curr_counts = curr_counts.join(replicate_list)

    average_of_transpose = curr_counts.groupby('sgRNA_pair')['FC_Averaged'].apply(np.nanmean)
    curr_counts = curr_counts.join(average_of_transpose,
                             on = 'sgRNA_pair',
                             rsuffix = "_abbaAveraged")
    
    return(curr_counts)


def prepare_study_for_export(sequence_ref, counts_ref, score_ref, study_controls = None, study_conditions = None, can_control_be_substring = True, remove_unrelated_counts = False):
    '''
        
    Prepares the counts, scores, and sequences files for insertion into the DB.

    **Params**:

    * score_ref: A pandas table that adheres to the scores table template. 
    * sequence_ref: A pandas table that adheres to the sequence table template (default: None). 
    * counts_ref: A pandas table that adheres to the counts table template (default: None). 
    * study_controls: A list of control targets of the sgRNAs (default: None).
    * study_conditions: A list of two lists; first list contains the replicate names of initial time point, and second list contains the same for final time point (default: None).
    * can_control_be_substring: Can the controls be a substring of gene targets (in case of possible name conventions: default: True)
    * remove_unrelated_counts = Remove dual counts with targets that are outside of supplied scores targets? (default: False)

    **Returns**:

    * A dictionary of three items:
        * scores_ref: Contains the procesed scores table (if supplied)
        * sequences_ref: Contains the procesed sequences table (if supplied)
        * counts_ref: Contains the procesed counts table (if supplied)
    '''
    ## make sure the columns are within each table, if not return error
    sequence_ref_needed_columns = {'sgRNA_guide_name', 'sgRNA_guide_seq', 'sgRNA_target_name'}
    
    if sequence_ref is not None:
        if len(sequence_ref_needed_columns.difference(sequence_ref.columns)) > 0:
            print('Error')
            print('sequence_ref')
            return
        # reset index by default
        sequence_ref.sort_values('sgRNA_target_name', ignore_index = True, inplace = True)
        sequence_ref.reset_index(drop = True, inplace = True)
    
    counts_ref_needed_columns = {'guide_1', 'guide_2', 'gene_1', 'gene_2', 'count_replicates', 'cell_line_origin', 'study_origin', 'study_conditions'}
    if counts_ref is not None:
        if len(counts_ref_needed_columns.difference(counts_ref.columns)) > 0:
            print('Error')
            print('counts_ref')
            return
        # reset index by default
        counts_ref.reset_index(drop = True, inplace = True)
    
    score_ref_needed_columns = {'gene_1', 'gene_2', 'study_origin', 'cell_line_origin', 'SL_score', 'SL_score_cutoff', 'statistical_score', 'statistical_score_cutoff'}
    if (score_ref is None) and (counts_ref is not None):
        print('There are no scores, but there are counts...Generating Placeholder...')
        score_ref = create_placeholder_scores(counts_ref.copy(), sequence_ref.copy())
    if len(score_ref_needed_columns.difference(score_ref.columns)) > 0:
        print('Error')
        print('score_ref')
        return
    # reset index by default
    score_ref.reset_index(drop = True, inplace = True)
    
    if study_controls is not None:
        study_controls = [i.upper() for i in study_controls]
    
    
    ## prepare each table to be inserted to their respective tables
    
    print("Starting processing...")
    
    ################################# first, handle the scores ref
    print('Score reference...')
    
    # fill NA
    score_ref = score_ref.fillna(0)
    
    for col in ['gene_1', 'gene_2', 'cell_line_origin']:
        score_ref[col] = [i.upper() for i in score_ref[col]]
    
    genes_A = score_ref['gene_1'].values
    genes_B = score_ref['gene_2'].values
    sorted_genes = []
    for i in range(score_ref.shape[0]):
        sorted_genes.append('_'.join(sorted([genes_A[i], genes_B[i]])))
    score_ref["gene_pair"] = sorted_genes

    # remove same ones
    score_ref = score_ref.loc[~(score_ref["gene_1"].values == score_ref["gene_2"].values)]

    # remove controls from SL scores
    if study_controls is not None:
        control_idx = np.array([False] * score_ref.shape[0])

        for curr_control in study_controls:
            
            if can_control_be_substring:
                control_idx = control_idx | np.array([True if curr_control in i else False for i in score_ref["gene_1"]]) | np.array([True if curr_control in i else False for i in score_ref["gene_2"]])
                
            control_idx = control_idx | np.array([True if i in curr_control else False for i in score_ref["gene_1"]]) | np.array([True if i in curr_control else False for i in score_ref["gene_2"]])

        print('Controls within SL score that are removed: ')
        print(control_idx.sum())
        print('---')

        score_ref = score_ref.loc[~control_idx]
        
    if (score_ref['statistical_score_cutoff'].iloc[0] != 0) and (score_ref['SL_score_cutoff'].iloc[0] != 0):
        print('Both GI and Stat cutoffs are present...')
        score_ref['SL_or_not'] = (score_ref['SL_score'] <= (score_ref['SL_score_cutoff'].iloc[0])) & (score_ref['statistical_score'] <= (score_ref['statistical_score_cutoff'].iloc[0]))
    elif score_ref['SL_score_cutoff'].iloc[0] != 0:
        print('Only GI cutoff is present...')
        score_ref['SL_or_not'] = score_ref['SL_score'] <= score_ref['SL_score_cutoff'].iloc[0]
    elif score_ref['statistical_score'].iloc[0] != 0:
        print('Only Stat cutoff is present...')
        score_ref['SL_or_not'] = score_ref['statistical_score'] <= score_ref['statistical_score_cutoff'].iloc[0]
    else:
        print('No scores/stats cutoffs are available, possibly generated. Setting all to be NOT SL')
        score_ref['SL_or_not'] = [False] * score_ref.shape[0]
        
    score_ref.loc[score_ref['SL_or_not'], 'SL_or_not'] = 'SL'
    score_ref.loc[score_ref['SL_or_not'] != 'SL', 'SL_or_not'] = 'Not SL'
    
    ################################# score ref - DONE
    
    print('Counts reference...')
    
    if counts_ref is not None:

        for col in ['guide_1', 'guide_2', 'gene_1', 'gene_2', 'cell_line_origin']:
            counts_ref[col] = [i.upper() for i in counts_ref[col]]


        # label whether single, double, or control
        sgRNA_true_pair_index = np.array([i for i in range(counts_ref.shape[0]) if (str(counts_ref["gene_1"].iloc[i]) not in study_controls) and (str(counts_ref["gene_2"].iloc[i]) not in study_controls) and (str(counts_ref["gene_1"].iloc[i]) != str(counts_ref["gene_2"].iloc[i]))])
        print(' '.join(["Number of double pairs:", str(len(sgRNA_true_pair_index))]))

        sgRNA_control_pair_index = np.array([i for i in range(counts_ref.shape[0]) if (str(counts_ref["gene_1"].iloc[i]) in study_controls) and (str(counts_ref["gene_2"].iloc[i]) in study_controls)])
        print(' '.join(["Number of controls:", str(len(sgRNA_control_pair_index))]))

        sgRNA_single_gene_index = np.array(sorted(list(set(range(counts_ref.shape[0])).difference(set(np.concatenate((sgRNA_true_pair_index, sgRNA_control_pair_index)))))))
        print(' '.join(["Number of singles:", str(len(sgRNA_single_gene_index))]))

        if (len(sgRNA_single_gene_index) + len(sgRNA_control_pair_index) + len(sgRNA_true_pair_index)) != counts_ref.shape[0]:
            print('Missing annotation')
            print(counts_ref.shape[0] - (len(sgRNA_single_gene_index) + len(sgRNA_control_pair_index) + len(sgRNA_true_pair_index)))

        counts_ref['target_type'] = 'N/A'
        counts_ref['target_type'].iloc[sgRNA_true_pair_index] = 'Dual'
        counts_ref['target_type'].iloc[sgRNA_control_pair_index] = 'Control'
        counts_ref['target_type'].iloc[sgRNA_single_gene_index] = 'Single'


        if 'Type' in counts_ref.columns:
            counts_ref = counts_ref.drop(columns = ['Type'])
        if 'Sequencing' in counts_ref.columns:
            counts_ref = counts_ref.drop(columns = ['Sequencing'])


        ## seperate the replicate counts across T0 and TEnd
        counts_ref['T0_counts'] = ""
        counts_ref['T0_replicate_names'] = ""
        counts_ref['TEnd_counts'] = ""
        counts_ref['TEnd_replicate_names'] = ""

        if isinstance(study_conditions, dict):
            # for different cell_line_origins within a study

            for cell_line_origin in study_conditions:
                curr_conditions = study_conditions[cell_line_origin]

                # access the cell_line_origin counts
                access_level = counts_ref.loc[counts_ref['cell_line_origin'] == cell_line_origin].copy()

                ## get all conditions
                condition = access_level['study_conditions'].value_counts().index.tolist()
                condition = condition[0].split(';')

                # time point T_0
                t_0_index = np.array([i for i in range(len(condition)) if condition[i] in curr_conditions[0]])
                # time point T_end
                t_end_index = np.array([i for i in range(len(condition)) if condition[i] in curr_conditions[1]])

                # get counts
                replicate_sep =  access_level["count_replicates"].apply(    
                    lambda x: np.array(x.split(";"), dtype = np.float64)
                )

                # get time point 
                t_0_comb = replicate_sep.apply(
                    lambda x: check_repeated_constructs(x, t_0_index)
                ).apply(lambda x: ';'.join(x.astype(np.str_)))

                t_end_comb = replicate_sep.apply(
                    lambda x: check_repeated_constructs(x, t_end_index)#np.median(x[t_end_index])
                ).apply(lambda x: ';'.join(x.astype(np.str_)))

                access_level['T0_counts'] = t_0_comb
                access_level['T0_replicate_names'] = ';'.join(curr_conditions[0])
                access_level['TEnd_counts'] = t_end_comb
                access_level['TEnd_replicate_names'] = ';'.join(curr_conditions[1])

                counts_ref.loc[counts_ref['cell_line_origin'] == cell_line_origin] = access_level
        else:
            # for only one cell_line_origin
            curr_conditions = study_conditions

            ## get all conditions
            condition = counts_ref['study_conditions'].value_counts().index.tolist()
            condition = condition[0].split(';')

            # time point T_0
            t_0_index = np.array([i for i in range(len(condition)) if condition[i] in curr_conditions[0]])
            # time point T_end
            t_end_index = np.array([i for i in range(len(condition)) if condition[i] in curr_conditions[1]])

            # get counts
            replicate_sep =  counts_ref["count_replicates"].apply(    
                lambda x: np.array(x.split(";"), dtype = np.float64)
            )

            # get time point 
            t_0_comb = replicate_sep.apply(
                lambda x: check_repeated_constructs(x, t_0_index)
            ).apply(lambda x: ';'.join(x.astype(np.str_)))

            t_end_comb = replicate_sep.apply(
                lambda x: check_repeated_constructs(x, t_end_index)#np.median(x[t_end_index])
            ).apply(lambda x: ';'.join(x.astype(np.str_)))

            counts_ref['T0_counts'] = t_0_comb
            counts_ref['T0_replicate_names'] = ';'.join(curr_conditions[0])
            counts_ref['TEnd_counts'] = t_end_comb
            counts_ref['TEnd_replicate_names'] = ';'.join(curr_conditions[1])


        # proceed to add the orientation

        unsorted_orientations = np.array(['|'.join([counts_ref["gene_1"].iloc[i], counts_ref["gene_2"].iloc[i]]) for i in range(counts_ref.shape[0])])
        sorted_orientations = np.array(['|'.join(sorted([counts_ref["gene_1"].iloc[i], counts_ref["gene_2"].iloc[i]])) for i in range(counts_ref.shape[0])])

        counts_ref['gene_pair'] = sorted_orientations
        counts_ref['gene_pair_orientation'] = 'A_B'
        counts_ref['gene_pair_orientation'].loc[sorted_orientations != unsorted_orientations] = 'B_A'
        
        counts_ref_list = []
        # applied for HORLBECK
        if remove_unrelated_counts:
            print('remove_unrelated_counts = TRUE')
            counts_ref_list = []
            for cl in sorted(list(set(counts_ref['cell_line_origin_origin']))):
                print('For cl = ' + cl)
                temp_counts = counts_ref.loc[counts_ref['cell_line_origin_origin'] == cl]
                temp_scores = score_ref.loc[score_ref['cell_line_origin_origin'] == cl]
                
                # get score genes + control
                score_genes = list(set(temp_scores['gene_1'].tolist() + temp_scores['gene_2'].tolist())) + study_controls
                if study_controls is not None:
                    score_genes = score_genes + study_controls
                # filter down
                temp_counts_filt = temp_counts.loc[(temp_counts['gene_1'].isin(score_genes) & temp_counts['gene_2'].isin(score_genes)) | (temp_counts['target_type'] == 'Control')]
                
                removed = temp_counts.shape[0] - temp_counts_filt.shape[0]
                if removed != 0:
                    print('Removed a total of {rem} sgRNAs...'.format(rem = removed))
                else:
                    print('No unrelated counts found!')
                    
                counts_ref_list.append(temp_counts_filt)
                
            counts_ref = pd.concat(counts_ref_list, axis = 0)
            counts_ref.reset_index(drop = True, inplace = True)

    ################################# counts ref - DONE
    
    print('Sequence reference...')
    if sequence_ref is not None:
        for col in ['sgRNA_guide_name', 'sgRNA_guide_seq', 'sgRNA_target_name']:
            sequence_ref[col] = [i.upper() for i in sequence_ref[col]]
            
        # set the target names to control
        control_idx = np.array([True if str(sequence_ref['sgRNA_target_name'].iloc[i]) in study_controls else False for i in range(sequence_ref.shape[0])]) | np.array([True if str(sequence_ref['sgRNA_guide_name'].iloc[i]) in study_controls else False for i in range(sequence_ref.shape[0])])
        sequence_ref.loc[control_idx, 'sgRNA_target_name'] = 'CONTROL'    
        # add study origin as well
        sequence_ref['study_origin'] = [counts_ref['study_origin'].iloc[0]] * sequence_ref.shape[0]
        
        sequence_ref.reset_index(drop=True, inplace = True)
    
    ################################# sequence ref - DONE
    
    print('Done! Returning...')
    return({'sequence_ref': sequence_ref,
            'counts_ref': counts_ref,
            'score_ref': score_ref})

def sort_pairs_and_guides(curr_counts):
    # sort the genes and guides based on gene ordering
    print('Sorting gene pairs and guides based on ordering gene ordering...')
    gene_pairs = []
    gene_pair_guides = []
    for i in range(curr_counts.shape[0]):

        guide_1 = curr_counts['sgRNA_guide_name_g1'].iloc[i]
        guide_2 = curr_counts['sgRNA_guide_name_g2'].iloc[i]

        gene_1 = curr_counts['sgRNA_target_name_g1'].iloc[i]
        gene_2 = curr_counts['sgRNA_target_name_g2'].iloc[i]

        t_gene_1, t_gene_2 = sorted([gene_1, gene_2])


        if (t_gene_1 == gene_1) and (t_gene_2 == gene_2):
            gene_1 = t_gene_1
            gene_2 = t_gene_2
        else:
            gene_1 = t_gene_1
            gene_2 = t_gene_2

            # swap the guides accordingly
            temp = guide_1
            guide_1 = guide_2
            guide_2 = temp

        gene_pairs.append('|'.join([gene_1, gene_2]))
        gene_pair_guides.append('|'.join([guide_1, guide_2]))

    return(gene_pairs, gene_pair_guides)

def reindex_alphbetically(df):
    result = []
    for index, row in df.iterrows():
        a, b = index.split('_')
        if a < b:
            result.append(f'{a}_{b}')
        else:
            result.append(f'{b}_{a}')
    
    
    return(result)

In [ ]:
import os
print(os.getcwd())

In [1]:
import os
print(os.getcwd())

C:\Users\Hamda\OneDrive\Documents\GitHub\PostDoc\Conway\Research\GIScoring\Hamda's Work\Horlbeck
